In [1]:
import gymnasium as gym
from rdquantum.hamiltonian import Rydberg_Cz

from ray.rllib.algorithms import ppo
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.logger import pretty_print
from ray.tune.registry import register_env

import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["figure.figsize"] = (10, 5)

In [2]:
H = Rydberg_Cz()
action = {
    'omega_p_amp': [100],
    'omega_r_amp': [175],
    'delta_p_amp': [400],
    'gate_time': [1.0]
}
fidelity = H.evaluation(action)
print(fidelity)

TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
prob, measurement_outcome, reward = H.measurement(action)
print(prob, measurement_outcome, reward)
print(type(prob), type(measurement_outcome), type(reward))

# Loading Environment

In [ ]:
# load environment
env = gym.make('rdquantum/HamiltonianTrainer-v2023.04.22', Hamiltonian=Rydberg_Cz)

def env_creator(env_config):
    return env  # return an env instance

register_env("HamiltonianTrainer-v2023.04.22", env_creator)

In [ ]:
algo = (
    PPOConfig()
    .framework(framework='torch')
    .training(train_batch_size=1, sgd_minibatch_size=1, num_sgd_iter=1)
    .rollouts(num_rollout_workers=1)
    .resources(num_gpus=0)
    # .environment(env='QubitStatePrep-v2023.04.11', disable_env_checking=True)
    .environment(env='HamiltonianTrainer-v2023.04.22')
    .build()
)
result = algo.train()
# episode_rewards = []
# for i in range(1):
#     result = algo.train()
#     episode_reward = np.asarray(result['hist_stats']['episode_reward'][-1:])
#     episode_rewards.append(episode_reward)

# #     if i % 5 == 0:
# #         checkpoint_dir = algo.save()
# #         print(f"Checkpoint saved in directory {checkpoint_dir}")
# episode_rewards = np.asarray(episode_rewards)

In [ ]:
plt.matshow(episode_rewards.transpose())
plt.xlabel("Epoch")
plt.ylabel("Episode")
plt.gca().invert_yaxis()
plt.gca().tick_params(axis="x", bottom=True, top=False, labelbottom=True, labeltop=False)
# plt.colorbar()

In [ ]:
result